*** Speaker diarization and transcription
1. Use Whisper model to get transcription and (start, end) pair
2. Segment the audio file into pieces according to (start, end) pair that we got from (1)
3. Do speaker embedding
4. Use {}

In [1]:
!pip install -U yt-dlp -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 44.7 MB/s eta 0:00:00


In [2]:
!wget -O - -q  https://github.com/yt-dlp/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz | xz -qdc| tar -x

In [3]:
# 부부 대화
!yt-dlp -xv --ffmpeg-location ffmpeg-master-latest-linux64-gpl/bin --audio-format wav  -o marriage_conversation.wav -- https://youtu.be/H0gH8orEidc

[debug] Command-line config: ['-xv', '--ffmpeg-location', 'ffmpeg-master-latest-linux64-gpl/bin', '--audio-format', 'wav', '-o', 'marriage_conversation.wav', '--', 'https://youtu.be/H0gH8orEidc']
[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out utf-8, error utf-8, screen utf-8
[debug] yt-dlp version stable@2023.03.04 [392389b7d] (pip)
[debug] Python 3.10.11 (CPython x86_64 64bit) - Linux-5.10.147+-x86_64-with-glibc2.31 (OpenSSL 1.1.1f  31 Mar 2020, glibc 2.31)
[debug] exe versions: ffmpeg N-110422-g7b2851b290-20230430 (setts), ffprobe N-110422-g7b2851b290-20230430
[debug] Optional libraries: Cryptodome-3.17, brotli-1.0.9, certifi-2022.12.07, mutagen-1.46.0, sqlite3-2.6.0, websockets-11.0.2
[debug] Proxy map: {'colab_language_server': '/usr/colab/bin/language_service'}
[debug] Loaded 1786 extractors
[youtube] Extracting URL: https://youtu.be/H0gH8orEidc
[youtube] H0gH8orEidc: Downloading webpage
[youtube] H0gH8orEidc: Downloading android player API JSON
[youtube] H0gH8orEidc: 

In [4]:
!pip install pydub -q

In [5]:
! pip uninstall --yes torch torchvision torchaudio 
! pip3 install torch torchvision torchaudio --quiet

Found existing installation: torch 2.0.0+cu118
Uninstalling torch-2.0.0+cu118:
  Successfully uninstalled torch-2.0.0+cu118
Found existing installation: torchvision 0.15.1+cu118
Uninstalling torchvision-0.15.1+cu118:
  Successfully uninstalled torchvision-0.15.1+cu118
Found existing installation: torchaudio 2.0.1+cu118
Uninstalling torchaudio-2.0.1+cu118:
  Successfully uninstalled torchaudio-2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null

In [7]:
! git clone https://github.com/RF5/simple-speaker-embedding.git

Cloning into 'simple-speaker-embedding'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 116 (delta 63), reused 70 (delta 28), pack-reused 0
Receiving objects: 100% (116/116), 1.22 MiB | 4.24 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [8]:
import whisper
import wave
import numpy as np
import torch

In [9]:
model = whisper.load_model("large-v2")

100%|█████████████████████████████████████| 2.87G/2.87G [00:55<00:00, 56.0MiB/s]


In [10]:
result = model.transcribe("marriage_conversation.wav")
segments = result["segments"]

In [11]:
print(segments)

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.4, 'text': ' 그냥 여보가 말을 좀 가끔씩 아직은 노력 많이 하지만 거칠게 할 때가 있어서', 'tokens': [50364, 11208, 8228, 7062, 1453, 39692, 6796, 10583, 30895, 24317, 22729, 2124, 49388, 8358, 23286, 3675, 45398, 1810, 8981, 7765, 1453, 27937, 50734], 'temperature': 0.0, 'avg_logprob': -0.4083606680643927, 'compression_ratio': 1.5132275132275133, 'no_speech_prob': 0.2872884273529053}, {'id': 1, 'seek': 0, 'start': 7.4, 'end': 9.040000000000001, 'text': ' 존댓말 쓰잖아요 그래서', 'tokens': [50734, 22116, 1327, 234, 241, 11023, 17373, 13928, 8844, 50816], 'temperature': 0.0, 'avg_logprob': -0.4083606680643927, 'compression_ratio': 1.5132275132275133, 'no_speech_prob': 0.2872884273529053}, {'id': 2, 'seek': 0, 'start': 9.040000000000001, 'end': 11.040000000000001, 'text': ' 거칠게 존댓말 하잖아요', 'tokens': [50816, 3675, 45398, 1810, 22116, 1327, 234, 241, 11023, 3369, 13928, 50916], 'temperature': 0.0, 'avg_logprob': -0.4083606680643927, 'compression_ratio': 1.5132275132275133, 'no_speech_prob

In [12]:
for count, x in enumerate(segments):
  print(x["text"])

 그냥 여보가 말을 좀 가끔씩 아직은 노력 많이 하지만 거칠게 할 때가 있어서
 존댓말 쓰잖아요 그래서
 거칠게 존댓말 하잖아요
 아니
 음
 음
 왜
 뭐가 있어요?
 힘줄?
 힘줄인가?
 소의 힘줄이지 뭐 뼈가 있겠어요?
 뼈도 있지
 그 비아냥거리는 말투도 싫어요
 어떻게 살아요 내랑


In [13]:
start_end = [(x["start"], x["end"]) for c, x in enumerate(segments)]

In [14]:
print(start_end)

[(0.0, 7.4), (7.4, 9.040000000000001), (9.040000000000001, 11.040000000000001), (11.040000000000001, 13.040000000000001), (13.040000000000001, 15.040000000000001), (15.040000000000001, 17.04), (17.04, 19.04), (19.04, 19.84), (19.84, 21.84), (21.84, 23.84), (23.84, 25.84), (25.84, 27.84), (28.68, 30.68), (32.68, 34.68)]


In [15]:
print(len(start_end))

14


In [16]:
import wave

def segment_wav_file(input_file_path, output_file_path, start_ms, end_ms):
    # Open the input WAV file
    with wave.open(input_file_path, 'rb') as input_wav:
        # Get the sample width (in bytes) and the number of channels
        sample_width = input_wav.getsampwidth()
        num_channels = input_wav.getnchannels()
        # Calculate the start and end positions (in frames)
        #start_pos = int(start_ms * input_wav.getframerate() / 1000)
        #end_pos = int(end_ms * input_wav.getframerate() / 1000)
        start_pos = int(start_ms * input_wav.getframerate() )
        end_pos = int(end_ms * input_wav.getframerate())
        # Calculate the number of frames to read
        num_frames = end_pos - start_pos
        # Set the parameters for the output WAV file
        output_wav = wave.open(output_file_path, 'wb')
        output_wav.setparams((num_channels, sample_width, input_wav.getframerate(), num_frames, input_wav.getcomptype(), input_wav.getcompname()))
        # Set the position in the input WAV file
        input_wav.setpos(start_pos)
        # Read the frames from the input WAV file and write them to the output WAV file
        output_wav.writeframes(input_wav.readframes(num_frames))
        # Close the output WAV file
        output_wav.close()

In [ ]:
#segment_wav_file(input_file_path, output_file_path, start_ms, end_ms)

In [17]:
count = 1
for start, end in start_end:
  # mill_secs has start and end pair
  # segment the WAV file into pieces
  segment_wav_file("marriage_conversation.wav", f"segmented{count}.wav", start, end)
  count += 1

In [19]:
 import torch
 # for the new model
 embedding_model = torch.hub.load('RF5/simple-speaker-embedding', 'convgru_embedder')
 # for the original model
 # model = torch.hub.load('RF5/simple-speaker-embedding', 'gru_embedder')
 embedding_model.eval()

Using cache found in /root/.cache/torch/hub/RF5_simple-speaker-embedding_master


ConvGRUEmbedder(
  (model): ConvRNNEmbedder(
    (conv_encoder): ConvEncoder(
      (conv_layers): ModuleList(
        (0): Sequential(
          (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GroupNorm(512, 512, eps=1e-05, affine=True)
          (3): GELU(approximate='none')
        )
        (1-4): 4 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
        (5-6): 2 x Sequential(
          (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (1): Dropout(p=0.0, inplace=False)
          (2): GELU(approximate='none')
        )
      )
    )
    (rnn): GRU(512, 768, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
    (head): Linear(in_features=1536, out_features=256, bias=True)
  )
)

In [20]:
#mel = model.melspec_from_file('marriage_conversation.wav')
#if you were using the `convgru_embedder`, you can go:
import librosa
wav1, _ = librosa.load('segmented1.wav', sr=16000)
wav1 = torch.from_numpy(wav1).float()

wav2, _ = librosa.load('segmented2.wav', sr=16000)
wav2 = torch.from_numpy(wav2).float()


e = [] # list of vectors of audio files

for count in range(len(start_end)):
  wav, _ = librosa.load(f'segmented{count + 1}.wav', sr=16000)
  wav = torch.from_numpy(wav).float()
  element = embedding_model(wav[None])
  e.append(element)

In [ ]:
# e1 = embedding_model(wav1[None]) # torch.Size([1, 256])
# e2 = embedding_model(wav2[None])

In [21]:
print(len(e))

14


In [22]:
print(e[0].shape)

torch.Size([1, 256])


In [23]:
from scipy.spatial.distance import cosine

In [24]:
e2 = [data.cpu().detach().numpy().flatten() for data in e] # flattened vectors

In [25]:
print(e2[0].shape)

(256,)


In [ ]:
print(cosine(e2[0], e2[1]))
print(cosine(e2[0], e2[2]))
print(cosine(e2[0], e2[3]))
print(cosine(e2[0], e2[4]))
print(cosine(e2[0], e2[5]))
print(cosine(e2[0], e2[6]))
print(cosine(e2[0], e2[7]))
print(cosine(e2[0], e2[8]))
print(cosine(e2[0], e2[9]))
print(cosine(e2[0], e2[10]))
print(cosine(e2[0], e2[11]))
print(cosine(e2[0], e2[12]))
print(cosine(e2[0], e2[13]))


0.8749587535858154
0.4259982109069824
0.8223230689764023
1.018939957022667
1.0134530663490295
1.08206807076931
0.5749915242195129
0.9546050019562244
0.813438892364502
1.0255341492593288
1.001039493829012
0.31542545557022095
0.9656992666423321


In [26]:
from sklearn.cluster import AgglomerativeClustering

In [27]:
clustering = AgglomerativeClustering(n_clusters=2).fit(e2)

In [28]:
print(clustering.labels_)

[1 0 1 0 0 0 0 1 0 0 0 0 1 0]


In [30]:
for count, x in enumerate(segments):
  print(f"speaker {clustering.labels_[count]} :", x["text"])

speaker 1 :  그냥 여보가 말을 좀 가끔씩 아직은 노력 많이 하지만 거칠게 할 때가 있어서
speaker 0 :  존댓말 쓰잖아요 그래서
speaker 1 :  거칠게 존댓말 하잖아요
speaker 0 :  아니
speaker 0 :  음
speaker 0 :  음
speaker 0 :  왜
speaker 1 :  뭐가 있어요?
speaker 0 :  힘줄?
speaker 0 :  힘줄인가?
speaker 0 :  소의 힘줄이지 뭐 뼈가 있겠어요?
speaker 0 :  뼈도 있지
speaker 1 :  그 비아냥거리는 말투도 싫어요
speaker 0 :  어떻게 살아요 내랑
